# Neural Style Transfer

In this project we trained a custom Neural Style Transfer (NST) model that can take a realisic content image and apply a style (painting).

In [1]:
import torch
import torch.nn as nn
import torch.optim as optimization
import numpy as np

import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.utils import save_image

from tqdm.notebook import tqdm
from PIL import Image

We'll crop the original images (content and style) to be at the same dimenstions. We'll define a crop function that crops 512x512 pixels from the center of each image:

In [2]:
def crop_it(img):
    image = Image.open(img)
    width, height = image.size
    left = (width-512)/2
    right = left+512
    top = (height-512)/2
    bottom = top+512
    cropped_img = image.crop((left, top, right, bottom))
    cropped_img.save("C:/Users/itayg/Documents/Portfolio/style_crop.jpg")
    return cropped_img.show

Let's apply it on each image:

In [3]:
crop_it('C:/Users/itayg/Documents/Portfolio/style.jpg')
crop_it('C:/Users/itayg/Documents/Portfolio/content.jpg')

<bound method Image.show of <PIL.Image.Image image mode=RGB size=512x512 at 0x2C5766FAF10>>

## Loss functions
### Content loss function

We'll implement a function to calculate the content loss which is the squared error loss between the two feature vectors of the content image and the target image.

In [ ]:
def get_content_loss(target_vec, content_vec):
  return torch.mean((target_vec-content_vec)**2)

### Style loss function

We'll implement a function to calculate the style loss by using Gram matrix. The total loss is the sum of every mean-squared distance (between two gram matrices of the style and the target images) for every layer times the weighted factor (the influence factor) of every layer.

In [ ]:
def gram_matrix(input, c, h, w):
  #c-channels; h-height; w-width 
  input = input.view(c, h*w) 
  #matrix multiplication on its own transposed form
  G = torch.mm(input, input.t())
  return G
  
def get_style_loss(target, style):
  _, c, h, w = target.size()
  G = gram_matrix(target, c, h, w) #gram matrix for the target image
  S = gram_matrix(style, c, h, w) #gram matrix for the style image
  return torch.mean((G-S)**2)/(c*h*w)

### The model (loading a pretrained VGG19 and modifying it)
We will use only 5 layers from the model (conv layers) just for feature extraction. We remove other layers used for classification.

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class VGG(nn.Module):
  def __init__(self):
    super(VGG, self).__init__()
    self.select_features = ['0', '5', '10', '19', '28'] #conv layers
    self.vgg = models.vgg19(pretrained=True).features
  
  def forward(self, output):
    features = []
    for name, layer in self.vgg._modules.items():
      output = layer(output)
      if name in self.select_features:
        features.append(output)
    return features

#load the model
vgg = VGG().to(device).eval()

In [ ]:
# Define the transformation
loader = transforms.Compose([
    transforms.Resize((512, 512)),  # Resize image
    transforms.ToTensor(),          # Convert image to tensor
])

Load image function:

In [5]:
def load_img(path):
  img = Image.open(path)
  img = loader(img).unsqueeze(0)
  return img.to(device)

In [7]:

# Define the transformation
loader = transforms.Compose([
    transforms.Resize((512, 512)),  # Resize image
    transforms.ToTensor(),          # Convert image to tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

content_img = load_img('C:/Users/itayg/Documents/portfolio/content_crop.jpg')
style_img = load_img('C:/Users/itayg/Documents/portfolio/style_crop.jpg')
#we can start from a random noise generated image or 
#just copy the content image as a starting point
target_img = content_img.clone().requires_grad_(True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
img_size = 512 if torch.cuda.is_available() else 128
#preprocessing of the images
loader = transforms.Compose([
                             transforms.Resize(img_size), 
                             transforms.ToTensor(),
                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

def load_img(path):
  img = Image.open(path)
  img = loader(img).unsqueeze(0)
  return img.to(device)

#initial model
model = models.vgg19(pretrained=True).features
#the optimizer: We use Adam since it's generally more adequate
optimizer = optimization.Adam([target_img], lr=0.001)

steps = 1000
alpha = 1 #content wight
beta = 10000 #style weight

#define the load_img function first
content_img = load_img('C:/Users/itayg/Documents/portfolio/content_crop.jpg')
style_img = load_img('C:/Users/itayg/Documents/portfolio/style_crop.jpg')
#we can start from a random noise generated image or 
#just copy the content image as a starting point
def ranom_noise(img):
 height, width = content_img.shape[:2]
 output_img = np.random.randn(height, width)
 return output_img


Loss calculation:

In [ ]:
#Function for saving and transforming the images:

def save(target, i):
  #the image needs to be denormalized first
  denormalization = transforms.Normalize((-2.12, -2.04, -1.80), (4.37, 4.46, 4.44))
  #remove the additional dimension
  img = target.clone().squeeze()
  img = denormalization(img).clamp(0, 1)
  save_image(img, f'result_{i}.png')

steps = 10000
target_img = content_img.clone().requires_grad_(True)
for step in tqdm(range(steps)):
  #get feature vectors representations for every image
  target_feature = vgg(target_img)
  content_feature = vgg(content_img)
  style_feature = vgg(style_img)

  #initiate the losses
  style_loss = 0
  content_loss = 0

  for target, content, style in zip(target_feature, content_feature, style_feature):
    content_loss += get_content_loss(target, content)
    style_loss += get_style_loss(target, style)
  
  #calculate the total loss 
  total_loss = alpha*content_loss+beta*style_loss
  
  #set parameters to zero
  optimizer.zero_grad()
  #compute the gradient
  total_loss.backward()
  #update parameters
  optimizer.step()

  #Let's follow the progress every 500 steps:
  if step%500 == 0:
    save(target_img, step)


In [ ]:
def save(target, i):
  #the image needs to be denormalized first
  denormalization = transforms.Normalize((-2.12, -2.04, -1.80), (4.37, 4.46, 4.44))
  #remove the additional dimension
  img = target.clone().squeeze()
  img = denormalization(img).clamp(0, 1)
  save_image(img, f'result_{i}.png')